In [24]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from src.data_loading.load_data import DataLoader


In [25]:
# Step 1: Load Data
file_path = '../data/raw/data.xlsx'  # Specify your file path here
data_loader = DataLoader(file_path)
df = data_loader.load_data()

Data loaded from ../data/raw/data.xlsx


In [26]:
# Step 2: Explore the data (summary statistics, missing values)
data_loader.explore_data()


Data Overview:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1761 entries, 0 to 1760
Data columns (total 7 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   Unnamed: 0                           1761 non-null   int64  
 1   Building's construction year         1761 non-null   int64  
 2   Number of floors                     1761 non-null   int64  
 3   Number of apartments                 1761 non-null   int64  
 4   Total electricity consumption (kWh)  1761 non-null   float64
 5   Longitude                            1761 non-null   float64
 6   Latitude                             1761 non-null   float64
dtypes: float64(3), int64(4)
memory usage: 96.4 KB
None

First few rows of the data:
   Unnamed: 0  Building's construction year  Number of floors  \
0         176                          1940                 4   
1         185                          1920                 2   
2   

In [27]:
# Step 3: Handle missing values
df = data_loader.handle_missing_values()



Missing values per column:
Unnamed: 0                             0
Building's construction year           0
Number of floors                       0
Number of apartments                   0
Total electricity consumption (kWh)    0
Longitude                              0
Latitude                               0
dtype: int64

Missing values handled (filled with median).


In [28]:
# Step 4: Remove duplicates
df = data_loader.remove_duplicates()



Number of duplicate rows: 0


In [29]:
# Step 5: Detect outliers (Z-score and IQR)
outliers_zscore = data_loader.detect_outliers_zscore()



Outliers detected using Z-score (|Z| > 3):
       Unnamed: 0  Building's construction year  Number of floors  \
142          907                          1950                 5   
163          978                          1960                10   
210         1578                          1950                 5   
389         3547                          1920                 1   
407         3904                          1940                 1   
420         3994                          1960                 4   
495         5462                          1990                18   
504         5551                          1950                 5   
515         5917                          1900                 3   
572         6440                          1995                16   
610         6603                          2000                16   
727         9272                          1960                 4   
791         9900                          1950                 2   
861

In [30]:
outliers_iqr = data_loader.detect_outliers_iqr()


Outliers detected using IQR method:
       Unnamed: 0  Building's construction year  Number of floors  \
44           367                          1970                 3   
71           438                          1950                 2   
115          712                          1960                 6   
137          890                          1980                10   
138          891                          1980                11   
...          ...                           ...               ...   
1719       19647                          1990                 9   
1721       19650                          1980                 6   
1722       19651                          1980                 9   
1730       19710                          1980                 7   
1756       21408                          1940                 1   

      Number of apartments  Total electricity consumption (kWh)  Longitude  \
44                       2                              232.391  35

In [31]:
# Step 6: Check skewness
skewness = data_loader.check_skewness()


Skewness for each numerical feature:
 Unnamed: 0                            -0.103201
Building's construction year          -8.790279
Number of floors                       1.543842
Number of apartments                   7.025745
Total electricity consumption (kWh)    5.699579
Longitude                             -1.224980
Latitude                              -0.635503
zscore                                 5.699579
dtype: float64


Skewness Interpretation:

Skewness values indicate how asymmetric the distribution of a feature is:

Positive skew (values > 1): The tail is on the right side of the distribution.

Negative skew (values < -1): The tail is on the left side of the distribution.

Approximately zero skew (values between -1 and 1): The distribution is roughly symmetrical.

From our results, we can observe the following:

Building's construction year: Skewness = -8.79. This indicates that the construction years are likely clustered around the more recent years (the tail is on the left side), and this will definitely need transformation.

Number of floors: Skewness = 1.54. This is positively skewed, meaning there are fewer buildings with a large number of floors (the tail is on the right).

Number of apartments: Skewness = 7.03. This is heavily positively skewed, indicating that most buildings have fewer apartments, with a few having a large number of apartments.

Total electricity consumption (kWh): Skewness = 5.70. Similar to the Number of apartments, this feature is highly positively skewed. It suggests that most buildings consume lower electricity compared to a few with extremely high consumption.

Longitude and Latitude: These features have negative skew values close to zero, so they are fairly symmetrical.

zscore: Skewness = 5.70 (which matches the skew of Total electricity consumption (kWh)), likely indicating that zscore is derived from this feature.

Therefore, we will handle skewness in feature engineering part. For features with positive skew (like Number of apartments, Total electricity consumption (kWh)), we can apply a log transformation to reduce the skewness. The log transformation compresses large values and spreads out small ones, leading to a more normal distribution. For highly skewed data (e.g., Building's construction year, Number of apartments), we can also use Box-Cox or Yeo-Johnson transformations (available in sklearn), which are more general and can handle negative values.